In [ ]:
import pandas as pd
pd.options.plotting.backend = 'plotly'
from aust_covid.inputs import get_base_vacc_data
from aust_covid.vaccination import get_vacc_data_masks, add_booster_data_to_vacc
from aust_covid.plotting import plot_full_vacc, plot_program_coverage

In [ ]:
vacc_df = get_base_vacc_data()
masks = get_vacc_data_masks(vacc_df)
vacc_df = add_booster_data_to_vacc(vacc_df, 7, 100)

In [ ]:
plot_full_vacc(masks['age_16_2dose'], vacc_df)

In [ ]:
plot_program_coverage({m: masks[m] for m in ['age_16_3dose', 'age_16_4dose', 'age_12_15_2dose', 'age_5_11_2dose']}, vacc_df)

In [ ]:
vacc_df[['adult booster', 'adult booster smooth']].plot(title='Cumulative boosters in ages 16+')

In [ ]:
vacc_df['incremental adult booster'].plot(title='Smoothed incremental additional doses by day')

In [ ]:
vacc_df['boosted in preceding'].plot(title=f'Vaccinated in preceding period')

In [ ]:
vacc_df['prop boosted in preceding'].plot(title=f'Proportion vaccinated in preceding period')